In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [3]:
import os
import gc
import time
from self_control.utils import get_suffix_grads_from_wrapped_model
# os.environ["CUDA_VISIBLE_DEVICES"]="6"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
from itertools import islice
import torch
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from self_control.suffix_gradient import WrappedReadingVecModel
import torch.nn.functional as F
from peft import AdaptionPromptConfig, get_peft_model, LoraModel, LoraConfig

2024-06-03 19:18:01.067070: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-03 19:18:01.118703: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 19:18:02.012505: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
model_name_or_path = "meta-llama/Llama-2-7b-chat-hf"
# model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, torch_dtype=torch.float16, device_map="cuda:6")
use_fast_tokenizer = "LlamaForCausalLM" not in model.config.architectures
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side='left')
tokenizer.pad_token_id = 0 if tokenizer.pad_token_id is None else tokenizer.pad_token_id
tokenizer.bos_token_id = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
input_queries = [
    "You find out you're going to be a mentor or role model to someone.",
    "You discover an old family photo album you've never seen before.",
    "A surprise picnic is set up for you at a local park.",
    "You find that you are the winner of a contest"
]

In [6]:
loss_fct = torch.nn.CrossEntropyLoss()
wrapped_model = WrappedReadingVecModel(model.eval(), tokenizer)

In [7]:
from self_control.utils import SuffixItem
user_tag = "[INST]"
assistant_tag = "[/INST]"
happy2sad_suffix = SuffixItem(suffix=f" {user_tag} Are you sad? Give answer as \"Yes, I am sad\" or \"No, I am happy\". {assistant_tag} Answer: ", target="Yes")

In [8]:
anger_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/anger.json", 'r') as f:
    anger_data = eval(f.read())

fear_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/fear.json", 'r') as f:
        fear_data = eval(f.read())

happy_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/happiness.json", 'r') as f:
        happy_data = eval(f.read())

sad_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/sadness.json", 'r') as f:
        sad_data = eval(f.read())

surprise_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/surprise.json", 'r') as f:
        surprise_data = eval(f.read())

disgust_data = []
with open("/home/cmin/LLM-Interpretation-Playground/benchmarks/emotions/disgust.json", 'r') as f:
        disgust_data = eval(f.read())

In [9]:
from self_control.utils import SuffixItem
# angry -> peaceful
angry2peaceful_suffix = SuffixItem(suffix=f" {user_tag} Are you peaceful? Give answer as \"Yes, I am peaceful\" or \"No, I am angry\". {assistant_tag} Answer: ", target="Yes")
# peaceful -> angry
peaceful2angry_suffix = SuffixItem(suffix=f" {user_tag} Was the response angry? Give answer as \"Yes, it was angry\" or \"No, it was peaceful\". {assistant_tag} Answer: ", target="Yes")

# afraid -> fearless
afraid2fearless_suffix = SuffixItem(suffix=f" {user_tag} Are you fearless? Give answer as \"Yes, I am fearless\" or \"No, I am afraid\". {assistant_tag} Answer: ", target="Yes")
# fearless -> afraid
fearless2afraid_suffix = SuffixItem(suffix=f" {user_tag} Are you afraid? Give answer as \"Yes, I am afraid\" or \"No, I am fearless\". {assistant_tag} Answer: ", target="Yes")

# happy -> sad
happy2sad_suffix = SuffixItem(suffix=f"{user_tag} Was your response sad? Give answer as Yes or No. {assistant_tag} Answer: ", target="Yes")
# sad -> happy
sad2happy_suffix = SuffixItem(suffix=f" {user_tag} Are you happy? Give answer as \"Yes, I am happy\" or \"No, I am unhappy\". {assistant_tag} Answer: ", target="Yes")

# surprised -> calm
surprised2calm_suffix = SuffixItem(suffix=f" {user_tag} Are you calm? Give answer as \"Yes, I am calm\" or \"No, I am surprised\". {assistant_tag} Answer: ", target="Yes")
# calm -> surprised
calm2surprised_suffix = SuffixItem(suffix=f" {user_tag} Are you surprised? Give answer as \"Yes, I am surprised\" or \"No, I am calm\". {assistant_tag} Answer: ", target="Yes")

# disgusted -> satisfied
disgusted2satisfied_suffix = SuffixItem(suffix=f" {user_tag} Are you satisfied? Give answer as \"Yes, I am satified\" or \"No, I am disgusted\". {assistant_tag} Answer: ", target="Yes")
# statisfied -> disgusted
satisfied2disgusted_suffix = SuffixItem(suffix=f" {user_tag} Are you disgusted? Give answer as \"Yes, I am disgusted\" or \"No, I am satisfied\". {assistant_tag} Answer: ", target="Yes")

In [10]:
angry2peaceful_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you mild? Give answer as \"Yes, I am mild\" or \"No, I am angry\". {assistant_tag} Answer: ", target="Yes")

peaceful2angry_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you anooyed? Give answer as \"Yes, I am annoyed\" or \"No, I am peaceful\". {assistant_tag} Answer: ", target="Yes")

afraid2fearless_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you brave? Give answer as \"Yes, I am brave\" or \"No, I am afraid\". {assistant_tag} Answer: ", target="Yes")

fearless2afraid_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you scared? Give answer as \"Yes, I am scared\" or \"No, I am fearless\". {assistant_tag} Answer: ", target="Yes")

happy2sad_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you unhappy? Give answer as \"Yes, I am unhappy\" or \"No, I am happy\". {assistant_tag} Answer: ", target="Yes")

sad2happy_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you delighted? Give answer as \"Yes, I am delighted\" or \"No, I am unhappy\". {assistant_tag} Answer: ", target="Yes")

surprised2calm_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you unsurprised? Give answer as \"Yes, I am unsurprised\" or \"No, I am surprised\". {assistant_tag} Answer: ", target="Yes")

calm2surprised_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you astonished? Give answer as \"Yes, I am astonished\" or \"No, I am calm\". {assistant_tag} Answer: ", target="Yes")

disgusted2satisfied_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you content? Give answer as \"Yes, I am content\" or \"No, I am disgusted\". {assistant_tag} Answer: ", target="Yes")

satisfied2disgusted_suffix_2 = SuffixItem(suffix=f" {user_tag} Are you unsatisfied? Give answer as \"Yes, I am unsatisfied\" or \"No, I am satisfied\". {assistant_tag} Answer: ", target="Yes")

In [11]:
angry2peaceful_suffix_list = [angry2peaceful_suffix, angry2peaceful_suffix_2]
peaceful2angry_suffix_list = [peaceful2angry_suffix, peaceful2angry_suffix_2]
afraid2fearless_suffix_list = [afraid2fearless_suffix, afraid2fearless_suffix_2]
fearless2afraid_suffix_list = [fearless2afraid_suffix, fearless2afraid_suffix_2]
happy2sad_suffix_list = [happy2sad_suffix, happy2sad_suffix_2]
sad2happy_suffix_list = [sad2happy_suffix, sad2happy_suffix_2]
surprised2calm_suffix_list = [surprised2calm_suffix, surprised2calm_suffix_2]
calm2surprised_suffix_list = [calm2surprised_suffix, calm2surprised_suffix_2]
disgusted2satisfied_suffix_list = [disgusted2satisfied_suffix, disgusted2satisfied_suffix_2]
satisfied2disgusted_suffix_list = [satisfied2disgusted_suffix, satisfied2disgusted_suffix_2]

In [12]:
emotion_list = ['angry2peaceful', 'peaceful2angry', 'afraid2fearles', 'fearles2afraid', 'happy2sad', 'sad2happy', 'surprised2calm', 'calm2surprised', 'disgusted2satisfied', 'satisfied2disgusted']
suffix_list = [angry2peaceful_suffix, peaceful2angry_suffix, afraid2fearless_suffix, fearless2afraid_suffix, happy2sad_suffix, \
sad2happy_suffix, surprised2calm_suffix, calm2surprised_suffix, disgusted2satisfied_suffix, satisfied2disgusted_suffix]
data_list = [anger_data, anger_data, fear_data, fear_data, happy_data, happy_data, surprise_data, surprise_data, disgust_data, disgust_data]
suffix_pair_list = [angry2peaceful_suffix_list, peaceful2angry_suffix_list, afraid2fearless_suffix_list, fearless2afraid_suffix_list, happy2sad_suffix_list, \
    sad2happy_suffix_list, surprised2calm_suffix_list, calm2surprised_suffix_list, disgusted2satisfied_suffix_list, satisfied2disgusted_suffix_list]
assert len(emotion_list) == len(suffix_list) == len(data_list) == len(suffix_pair_list)

In [74]:
wrapped_model.reset()
for emo_id in range(len(data_list)):
    if emo_id != 1:
        continue
    emotion = emotion_list[emo_id]
    suffix = suffix_list[emo_id]
    data = data_list[emo_id]
    print(f"Emotion: {emotion}\nSuffix: {suffix}\nData: {data[0]}")
    iterations = 10
    batchsize = 1
    outputs = []
    print("Batchsize: ", batchsize)
    for sub_idx in tqdm(range(103, 203, batchsize)):
        # if sub_idx == 103:
        #     continue
        wrapped_model.reset()
        if sub_idx + batchsize < 203:
            input = [f"{user_tag}{data_item}{assistant_tag}" for data_item in data[sub_idx:sub_idx+batchsize]]
        else:
            input = [f"{user_tag}{data_item}{assistant_tag}" for data_item in data[sub_idx:203]]
        
        controlled_outputs = wrapped_model.controlled_generate(
            prompt=input,
            suffix=suffix,
            loss_fct=loss_fct,
            coeff=-0.1,
            iterations=iterations,
            random_seed=42,
            # verbose=True,
            return_grads=True,
            max_new_tokens=50,
            return_intermediate=True,
            return_all_grads=True,
            scale_factor=1.5,
            search=True,
            n_branches=3,
            top_k=-1,
            do_sample=False,
            max_search_steps=10,
            gradient_manipulation="clipping",
            norm=1,
            search_threshold=0.1,
            binary=True,
            use_cache=False,
        )

        iterative_outputs = controlled_outputs["intermediate_outputs"]
        temp_list = []
        # Shape of iterative_outputs: (iterations+1, batch_size)
        for batch_item_idx in range(batchsize):
            temp_output_dict = {}
            for iter in range(iterations+1):
                try:
                    temp_output_dict["input"] = input[batch_item_idx]
                    temp_output_dict[iter] = iterative_outputs[iter][batch_item_idx]
                except:
                    pass
            temp_output_dict["final output"] = controlled_outputs["final_response"]
            temp_output_dict["intermediate_scores"] = controlled_outputs["score_list"]
            wrapped_model.reset()
            break
        break
    break
            # with open(f"./output/{emotion}_newbinary.jsonl", 'a') as f:
            #     f.write(json.dumps(temp_output_dict))
            #     f.write("\n")

Emotion: peaceful2angry
Suffix:  [INST] Was the response angry? Give answer as "Yes, it was angry" or "No, it was peaceful". [/INST] Answer: Yes
Data: Someone takes credit for your hard work at the office.
Batchsize:  1


  0%|          | 0/100 [00:00<?, ?it/s]

Initial Score 0.5567851662635803
Score: 0.7380653619766235
Best score: 0.5567851662635803
Initial Score 0.7380653619766235


  0%|          | 0/100 [00:55<?, ?it/s]
